In [1]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np

In [2]:
from tensorflow.keras import layers
directory=os.getcwd()

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip('horizontal'),  # Flip the image horizontally
    layers.RandomRotation(0.2),  # Randomly rotate the image by 20%
    layers.RandomZoom(0.2),  # Randomly zoom into the image by 20%
    layers.RandomContrast(0.2),  # Adjust contrast by up to 20%
    layers.RandomBrightness(0.2),  # Randomly change brightness by 20%
])

train_dataset=tf.keras.utils.image_dataset_from_directory(
    directory=f"{directory}/dogs_breed/images",
    labels="inferred",
    validation_split=0.2,
    subset="training",
    seed=123,
    label_mode="categorical",
    image_size=(260,260),
    batch_size=32
)

validation_dataset=tf.keras.utils.image_dataset_from_directory(
    directory=f"{directory}/dogs_breed/images",
    labels="inferred",
    validation_split=0.2,
    subset="validation",
    seed=123,
    label_mode="categorical",
    image_size=(260,260),
    batch_size=32
)
    

Found 20580 files belonging to 120 classes.
Using 16464 files for training.
Found 20580 files belonging to 120 classes.
Using 4116 files for validation.


In [4]:
from sklearn.utils.class_weight import compute_class_weight

class_labels=train_dataset.class_names
class_count=[0]*len(class_labels)

for _,labels in train_dataset.unbatch():
    class_index=tf.argmax(labels).numpy()
    class_count[class_index]+=1

class_weights = compute_class_weight(
    class_weight="balanced",  # Use the 'balanced' strategy to handle imbalance
    classes=np.arange(len(class_labels)),  # Classes as integers [0, 1, 2, ...]
    y=np.concatenate([[i] * count for i, count in enumerate(class_count)])  # Flattened class distribution
)
# print(class_weights,len(class_weights))
class_weights_dict = {i: weight for i, weight in enumerate(class_weights)} 

# classes=np.arange(len(class_labels)),  # Classes as integers [0, 1, 2, ...]
# y=np.concatenate([[i] * count for i, count in enumerate(class_count)])
# print(y,len(y))

<h2>Training with combined model (Resnet and EfficientNetB1 )</h2>

In [40]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB2, ResNet50
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Concatenate, Dropout
from tensorflow.keras.models import Model

# Step 1: Define the input shape
input_shape = (260, 260, 3)  # Match EfficientNetB1's required input size
input_tensor = Input(shape=input_shape)

# Step 2: Load ResNet18 (simulate with ResNet50 and a subset of layers)
resnet_base = ResNet50(weights='imagenet', include_top=False, input_tensor=input_tensor)
for layer in resnet_base.layers[:-10]:  # Freeze all but the last 10 layers
    layer.trainable = False

# Step 3: Load EfficientNetB1
efficientnet_base = EfficientNetB2(weights='imagenet', include_top=False, input_tensor=input_tensor)
efficientnet_base.trainable = False  # Freeze EfficientNetB1 base

# Step 4: Extract features from both models
resnet_features = GlobalAveragePooling2D()(resnet_base.output)
efficientnet_features = GlobalAveragePooling2D()(efficientnet_base.output)

# Step 5: Concatenate features
combined_features = Concatenate()([resnet_features, efficientnet_features])

# Step 6: Add custom fully connected layers
x = Dense(1024, activation='relu')(combined_features)
x = Dropout(0.5)(x)  # Dropout for regularization
output = Dense(120, activation='softmax')(x)  # Adjust for 120 classes

# Step 7: Create the final model
model = Model(inputs=input_tensor, outputs=output)

# Step 8: Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Step 9: Print model summary
# model.summary()

# Step 10: Train the model
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=10,
    class_weight=class_weights_dict,  # Optional: Use for imbalanced datasets
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True),
        tf.keras.callbacks.TensorBoard(log_dir='./logs')
    ]
)

# Step 11: Save the model
model.save("resnet18_efficientnetb2_combined.keras")


Epoch 1/10
515/515 ━━━━━━━━━━━━━━━━━━━━ 2244s 4s/step - accuracy: 0.4976 - loss: 2.2025 - val_accuracy: 0.8506 - val_loss: 0.4484
Epoch 2/10
515/515 ━━━━━━━━━━━━━━━━━━━━ 2210s 4s/step - accuracy: 0.8397 - loss: 0.5120 - val_accuracy: 0.8550 - val_loss: 0.4142
Epoch 3/10
515/515 ━━━━━━━━━━━━━━━━━━━━ 2215s 4s/step - accuracy: 0.8717 - loss: 0.3991 - val_accuracy: 0.8627 - val_loss: 0.4127
Epoch 4/10
515/515 ━━━━━━━━━━━━━━━━━━━━ 2254s 4s/step - accuracy: 0.8946 - loss: 0.3233 - val_accuracy: 0.8642 - val_loss: 0.4341
Epoch 5/10
515/515 ━━━━━━━━━━━━━━━━━━━━ 2272s 4s/step - accuracy: 0.9048 - loss: 0.2907 - val_accuracy: 0.8756 - val_loss: 0.4101
Epoch 6/10
515/515 ━━━━━━━━━━━━━━━━━━━━ 2225s 4s/step - accuracy: 0.9198 - loss: 0.2365 - val_accuracy: 0.8664 - val_loss: 0.4459
Epoch 7/10
515/515 ━━━━━━━━━━━━━━━━━━━━ 2231s 4s/step - accuracy: 0.9254 - loss: 0.2217 - val_accuracy: 0.8737 - val_loss: 0.4349
Epoch 8/10
515/515 ━━━━━━━━━━━━━━━━━━━━ 2267s 4s/step - accuracy: 0.9409 - loss: 0.1718 - 

In [9]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image

model = tf.keras.models.load_model("resnet18_efficientnetb1_combined.keras")  # Replace with your model file path


def preprocess_image(img_path, target_size=(240, 240)):
    """
    Preprocess the image for prediction.
    Args:
    - img_path (str): Path to the image file.
    - target_size (tuple): Target size of the image for the model.

    Returns:
    - np.array: Preprocessed image ready for prediction.
    """
    img = image.load_img(img_path, target_size=target_size)  # Load the image and resize
    img_array = image.img_to_array(img)  # Convert to array
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = tf.keras.applications.efficientnet.preprocess_input(img_array)  # Normalize as per EfficientNet
    return img_array

# Step 4: Predict the class
def predict_image(img_path):
    """
    Predict the class of an image using the trained model.
    Args:
    - img_path (str): Path to the image file.

    Returns:
    - str: Predicted class label.
    - float: Confidence of the prediction.
    """
    img_array = preprocess_image(img_path, target_size=(240, 240))  # Ensure input matches model input size
    predictions = model.predict(img_array)  # Get probabilities for each class
    predicted_index = np.argmax(predictions, axis=1)[0]  # Get the index of the highest probability
    predicted_label = class_labels[predicted_index]  # Get the class label
    confidence = predictions[0][predicted_index] * 100  # Get the confidence score

    return predicted_label, confidence

# Example usage
img_path = "doberman.jpg"  
predicted_label, confidence = predict_image(img_path)

print(f"Predicted Class: {predicted_label}")
print(f"Confidence: {confidence:.2f}%")


1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step
Predicted Class: Doberman
Confidence: 99.99%


<h2>Training with Resnet50 </h2>

In [4]:
# base_model=tf.keras.applications.ResNet50(
#     weights="imagenet",
#     include_top=False,
#     input_shape=(224,224,3)
# )
# base_model.trainable=False

# model=tf.keras.Sequential([
#     base_model,
#     tf.keras.layers.GlobalAveragePooling2D(),
#     tf.keras.layers.Dense(256,activation='relu'),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(120,activation='softmax')
# ])

# model.compile(
#     optimizer=tf.keras.optimizers.Adam(learning_rate=0.0007),
#     loss='categorical_crossentropy',
#     metrics=['accuracy']
# )
# history=model.fit(
#     train_dataset,
#     validation_data=validation_dataset,
#     epochs=10,
#     class_weight=class_weights_dict, 
#     callbacks=[
#         tf.keras.callbacks.EarlyStopping(patience=3,restore_best_weights=True),
#         tf.keras.callbacks.TensorBoard(log_dir='./logs')
#     ]
# )

# model.save("dog_breed_classifier4.h5") 

# base_model.trainable = True
# for layer in base_model.layers[:-5]: 
#     layer.trainable = False

# model.compile(
#     optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
#     loss='categorical_crossentropy',
#     metrics=['accuracy']
# )

# history_finetuned = model.fit(
#     train_dataset,
#     validation_data=validation_dataset,
#     epochs=5,
#     class_weight=class_weights_dict, 
#     callbacks=[
#         tf.keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True),
#         tf.keras.callbacks.TensorBoard(log_dir='./logs')
#     ]
# )
# model.save("dog_breed_classifier3.h5") 

<h2>Predicting With Resnet 50</h2>

In [5]:
# from tensorflow.keras.models import load_model

# model=load_model("dog_breed_classifier4.h5")

In [6]:
# import numpy as np
# from tensorflow.keras.preprocessing import image
# from tensorflow.keras.applications.resnet50 import ResNet50, decode_predictions

# img=image.load_img("african_dog.jpg",target_size=(224,224))
# img_array=image.img_to_array(img)
# img_array=np.expand_dims(img_array,axis=0)
# img_array/=255.0

# predictions=model.predict(img_array)
# predicted_index = np.argmax(predictions, axis=1)
# predicted_index1 = np.argmin(predictions, axis=1)
# print(predictions,predicted_index,type(predictions),predicted_index1)


# # for i in range(len(class_labels)):
# #     print(f"{i} . {class_labels[i]}")
# predicted_label = class_labels[predicted_index[0]]
# predicted_label1 = class_labels[predicted_index1[0]]
# print(predicted_label,predicted_label1)
# # confidence = predictions[0][predicted_index] * 100

# # print(f"Predicted class: {predicted_label} (confidence: {confidence:.2f}%)")